# Intro
Adaptation of `fastai` and `sklearn` libraries for Machine learning models using Kaggle 'buldozers' dataset.

For `fastai` implementation used documentation https://docs.fast.ai/tutorial.tabular.html and https://docs.fast.ai/tabular.core.html

# Prepare

## Imports

In [ ]:
%load_ext autoreload
# %autoreload 2
%matplotlib inline
# these lines let us edit modules code and this will automatically reload modules.

In [ ]:
from fastai.imports import *
from fastai.tabular import *
from fastai.tabular.core import * # not work on fastai 2.0.17
from fastai.tabular.all import * # not work on fastai 2.0.17

# from pandas_summary import DataFrameSummary
from sklearn.ensemble import RandomForestRegressor
from IPython.display import display

from sklearn import metrics

### If got errors importing fastai - try install version 2

if `fastai.tabular.core` or `fastai.tabular.all` give errors - this means you have wrong version of `fastai` library. Try uninstall and install it again.

In [ ]:
# !pip install --upgrade fastai
!pip uninstall fastai 

Uninstalling fastai-2.0.17:
  Would remove:
    /usr/local/lib/python3.6/dist-packages/fastai-2.0.17.dist-info/*
    /usr/local/lib/python3.6/dist-packages/fastai/*
Proceed (y/n)? y
  Successfully uninstalled fastai-2.0.17


In [ ]:
!pip install fastai #==2.0.17

     |████████████████████████████████| 194kB 10.2MB/s 
     |████████████████████████████████| 12.8MB 14.5MB/s 
     |████████████████████████████████| 776.7MB 22kB/s 
  Found existing installation: torch 1.6.0
    Uninstalling torch-1.6.0:
      Successfully uninstalled torch-1.6.0
  Found existing installation: torchvision 0.7.0
    Uninstalling torchvision-0.7.0:
      Successfully uninstalled torchvision-0.7.0


In [ ]:
!ls /usr/local/lib/python3.6/dist-packages/fastai

basics.py	imports.py    losses.py     _pytorch_doc.py  torch_imports.py
callback	__init__.py   medical	    tabular	     vision
collab.py	interpret.py  metrics.py    test_utils.py
data		launch.py     _nbdev.py     text
distributed.py	layers.py     optimizer.py  torch_basics.py
fp16_utils.py	learner.py    __pycache__   torch_core.py


In [ ]:
!cat /usr/local/lib/python3.6/dist-packages/fastai/version.py

__all__ = ['__version__']
__version__ = '1.0.61'


version 2 should have text with version number in file below

In [ ]:
!cat /usr/local/lib/python3.6/dist-packages/fastai/__init__.py

__version__ = "2.0.17"



In [ ]:
!ls /usr/local/lib/python3.6/dist-packages/fastai/tabular/
# !head 20 /usr/local/lib/python3.6/dist-packages/fastai/tabular/

all.py	core.py  data.py  __init__.py  learner.py  model.py  __pycache__


### mount google drive

In [ ]:
# google drive if needed
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


### Load buldozer data
You can download `Train.csv` from kaggle and then upload it to google drive.

In [ ]:
!ls  'drive/My Drive/Colab Notebooks/AIacademy/class-C/fast_ai/'

 data			       functions.py	        syntetic_data.ipynb
 fast-ai-random-forest.ipynb  'mnist fastai-v2.ipynb'   Test.csv
 fastai-v2-buldozers.ipynb    'Neural net.ipynb'        Train.csv


In [ ]:
buldozer_path = 'drive/My Drive/Colab Notebooks/AIacademy/class-C/fast_ai/'

In [ ]:
!head -2 'drive/My Drive/Colab Notebooks/AIacademy/class-C/fast_ai/Train.csv'

SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
1139246,66000,999089,3157,121,3,2004,68,Low,11/16/2006 0:00,521D,521,D,,,,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,,EROPS w AC,None or Unspecified,,None or Unspecified,,,,,,,,2 Valve,,,,,None or Unspecified,None or Unspecified,,,,,,,,,,,,,Standard,Conventional


Load from csv without optimization of memory usage by defining column types and sizes. Define only dates.

In [ ]:
%%time
df_raw = pd.read_csv(f'{buldozer_path}Train.csv', 
                     low_memory=False, parse_dates=["saledate","YearMade"])

CPU times: user 3.58 s, sys: 549 ms, total: 4.13 s
Wall time: 4.3 s


## First Exploration

In [ ]:
df_raw.shape

(401125, 53)

In [ ]:
df_raw.dtypes

SalesID                              int64
SalePrice                            int64
MachineID                            int64
ModelID                              int64
datasource                           int64
auctioneerID                       float64
YearMade                            object
MachineHoursCurrentMeter           float64
UsageBand                           object
saledate                    datetime64[ns]
fiModelDesc                         object
fiBaseModel                         object
fiSecondaryDesc                     object
fiModelSeries                       object
fiModelDescriptor                   object
ProductSize                         object
fiProductClassDesc                  object
state                               object
ProductGroup                        object
ProductGroupDesc                    object
Drive_System                        object
Enclosure                           object
Forks                               object
Pad_Type   

In [ ]:
df_raw.dtypes.value_counts()

object            45
int64              5
float64            2
datetime64[ns]     1
dtype: int64

## First transformations

### Log Y values 'salesPrice'

In [ ]:
df_raw.SalePrice.iloc[:3]

0    66000
1    57000
2    10000
Name: SalePrice, dtype: int64

In [ ]:
df_raw['SalePrice'] = np.log(df_raw.SalePrice)
df_raw.SalePrice.iloc[:3]

0    11.097410
1    10.950807
2     9.210340
Name: SalePrice, dtype: float64

### FastAI `shrink` dataframe
minimize column data types and convert `object` columns into categories (kind of `train_cats`)

In [ ]:
# fastai.tabular.core
%time df = df_shrink(df_raw, skip=['YearMade','saledate'])

CPU times: user 964 ms, sys: 399 ms, total: 1.36 s
Wall time: 1.37 s


below is check for the types of columns.

In [ ]:
# df_raw.dtypes
df_raw.value_counts()

Series([], dtype: int64)

In [ ]:
# df.dtypes
df.value_counts()

Series([], dtype: int64)

### Find categories 'object' columns for `fastAI` processing

In [ ]:
cont_names, cat_names = cont_cat_split(df_raw)
print(cont_names)
print(cat_names)

['SalesID', 'SalePrice', 'MachineID', 'ModelID', 'auctioneerID', 'MachineHoursCurrentMeter']
['datasource', 'YearMade', 'UsageBand', 'saledate', 'fiModelDesc', 'fiBaseModel', 'fiSecondaryDesc', 'fiModelSeries', 'fiModelDescriptor', 'ProductSize', 'fiProductClassDesc', 'state', 'ProductGroup', 'ProductGroupDesc', 'Drive_System', 'Enclosure', 'Forks', 'Pad_Type', 'Ride_Control', 'Stick', 'Transmission', 'Turbocharged', 'Blade_Extension', 'Blade_Width', 'Enclosure_Type', 'Engine_Horsepower', 'Hydraulics', 'Pushblock', 'Ripper', 'Scarifier', 'Tip_Control', 'Tire_Size', 'Coupler', 'Coupler_System', 'Grouser_Tracks', 'Hydraulics_Flow', 'Track_Type', 'Undercarriage_Pad_Width', 'Stick_Length', 'Thumb', 'Pattern_Changer', 'Grouser_Type', 'Backhoe_Mounting', 'Blade_Type', 'Travel_Controls', 'Differential_Type', 'Steering_Controls']


Normalize is removed as we will do `RandomForest` not neural nets.

In [ ]:
procs = [Categorify, FillMissing] #, Normalize]

# Data splits

## Manual split: dependend / independend

In [ ]:
df_train, df_test = df.iloc[:300000].copy(), df.iloc[300000:].copy()

In [ ]:
df_test.SalePrice.iloc[:3]

300000    10.434115
300001     9.798127
300002    10.146434
Name: SalePrice, dtype: float32

In [ ]:
Y_train = df_train.SalePrice
x_train = df_train.drop('SalePrice', axis=1)

In [ ]:
Y_test = df_test.SalePrice
x_test = df_test.drop('SalePrice', axis=1)

In [ ]:
Y_train.head(3)

0    11.097410
1    10.950807
2     9.210340
Name: SalePrice, dtype: float32

## SKlearn split

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
# TODO need to test
train_X, val_X, train_y, val_y = train_test_split(X, y, random_state = 0)

In [ ]:
type(df_train)

pandas.core.frame.DataFrame

In [ ]:
df_train.head(3)

,SalesID,SalePrice,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
0,1139246,11.097410,999089,3157,121,3.0,2004,68.0,Low,2006-11-16,521D,521,D,NaN,NaN,NaN,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
1,1139248,10.950807,117657,77,121,3.0,1996,4640.0,Low,2004-03-26,950FII,950,F,II,NaN,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,NaN,EROPS w AC,None or Unspecified,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,NaN,NaN,NaN,23.5,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Standard,Conventional
2,1139249,9.210340,434808,7009,121,3.0,2001,2838.0,High,2004-02-26,226,226,NaN,NaN,NaN,NaN,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,NaN,OROPS,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Auxiliary,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,Standard,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df_test.drop(['SalePrice'], axis=1, inplace=True)

In [ ]:
df_test.head(3)

,SalesID,MachineID,ModelID,datasource,auctioneerID,YearMade,MachineHoursCurrentMeter,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls
300000,2241104,285987,13776,136,NaN,2000,0.0,NaN,2007-05-11,D3CIIIXL,D3,C,III,XL,NaN,"Track Type Tractor, Dozer - 20.0 to 75.0 Horsepower",Virginia,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Hydrostatic,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,None or Unspecified,None or Unspecified,NaN,NaN
300001,2241111,115917,13776,136,1.0,2001,0.0,NaN,2010-06-30,D3CIIIXL,D3,C,III,XL,NaN,"Track Type Tractor, Dozer - 20.0 to 75.0 Horsepower",North Carolina,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN
300002,2241112,1714094,13776,136,NaN,1999,0.0,NaN,2007-07-14,D3CIIIXL,D3,C,III,XL,NaN,"Track Type Tractor, Dozer - 20.0 to 75.0 Horsepower",New York,TTT,Track Type Tractors,NaN,OROPS,NaN,NaN,NaN,NaN,Standard,NaN,NaN,NaN,NaN,NaN,2 Valve,NaN,None or Unspecified,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,None or Unspecified,PAT,None or Unspecified,NaN,NaN


## Fast AI

In [ ]:
# splits = RandomSplitter()(range_of(df_train))
splits = RandomSplitter(valid_pct=0.2)(range_of(df_raw))

NameError: ignored

In [ ]:
type(splits)

tuple

In [ ]:
splits[:3]

((#320900) [26011,75166,37736,142749,33174,389369,393453,167697,102512,30346...],
 (#80225) [197738,278728,196100,62413,157552,309314,164957,92725,310410,393560...])

In [ ]:
df_train.dtypes.value_counts()

object            45
int64              5
float64            2
datetime64[ns]     1
dtype: int64

### TO object

There are different spliters in `fastai`. Here I choose `TrainTestSplitter` but not sure it takes latest sorted rows by date.

In [ ]:
splits = TrainTestSplitter(test_size=0.3)(range_of(df))

In [ ]:
splits[0]

(#280787) [150144,163729,392636,91666,258321,361691,307627,377446,237386,89548...]

In [ ]:
%%time
to = TabularPandas(df, procs, cat_names, cont_names, y_names="SalePrice", splits=splits)

CPU times: user 1.59 s, sys: 90 ms, total: 1.68 s
Wall time: 1.7 s


In [ ]:
to.xs.iloc[:2]

,datasource,YearMade,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,SalePrice,MachineID,ModelID,auctioneerID,MachineHoursCurrentMeter
150144,2,53,0,2676,557,186,0,0,64,3,14,23,4,4,0,3,0,0,0,0,0,0,0,0,0,0,4,0,0,0,0,0,2,0,0,0,2,19,27,3,2,1,0,0,0,0,0,1,2,1524231,10.373491,1427198,1177,1.0,0.0
163729,2,40,0,3157,1242,368,15,0,0,0,35,24,2,2,3,1,0,0,0,0,5,0,1,6,3,1,5,1,2,2,2,17,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,2,1586430,9.350102,1527409,4777,8.0,0.0


In [ ]:
to.train

        SalesID  SalePrice  ...  auctioneerID_na  MachineHoursCurrentMeter_na
150144  1524231  10.373491  ...                1                            2
163729  1586430   9.350102  ...                1                            2
392636  6257981  10.985292  ...                1                            2
91666   1401514  11.170435  ...                1                            2
258321  1790589   9.104980  ...                1                            2
...         ...        ...  ...              ...                          ...
190089  1626612  10.518673  ...                1                            2
387807  4353671  11.434964  ...                1                            1
397366  6286309  10.146434  ...                1                            2
21988   1215306  10.596635  ...                1                            1
139433  1500926   9.546813  ...                1                            2

[280787 rows x 55 columns]

In [ ]:
to.ys.describe

<bound method NDFrame.describe of         SalePrice
368515  -0.851633
232517  -0.358788
355574  -1.180124
180215   0.147972
288734  -0.799185
...           ...
27327   -1.518793
364904  -0.436759
307504   1.358389
103274   0.947356
225984  -0.652403

[401125 rows x 1 columns]>

In [ ]:
dls = to.dataloaders()
dls.valid.show_batch()
# type(to)

,datasource,YearMade,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,SalePrice,SalePrice,MachineID,ModelID,auctioneerID,MachineHoursCurrentMeter,SalePrice,SalePrice
0,132,2003,#na#,2008-06-19,210LE,210,LE,#na#,#na#,Compact,Wheel Loader - 60.0 to 80.0 Horsepower,Arizona,WL,Wheel Loader,#na#,OROPS,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,True,1636008.0,16000.0,16000.0,1465800.0,4579.0,1.0,0.0,16000.0,16000.0
1,132,1000,#na#,1999-05-04,C14B,C14,B,#na#,#na#,Small,"Hydraulic Excavator, Track - 16.0 to 19.0 Metric Tons",Wisconsin,TEX,Track Excavators,#na#,EROPS,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,None or Unspecified,#na#,#na#,#na#,Steel,30 inch,None or Unspecified,None or Unspecified,None or Unspecified,Double,#na#,#na#,#na#,#na#,#na#,False,True,1844791.0,9000.0,9000.0,1479368.0,5847.0,6.0,0.0,9000.0,9000.0
2,132,1999,#na#,2006-08-30,653G,653,G,#na#,#na#,Large / Medium,"Hydraulic Excavator, Track - 19.0 to 21.0 Metric Tons",Alabama,TEX,Track Excavators,#na#,EROPS,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,None or Unspecified,#na#,#na#,#na#,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,#na#,#na#,#na#,#na#,#na#,False,True,1634006.0,33000.0,33000.0,1423637.0,4768.0,7.0,0.0,33000.0,33000.0
3,132,1983,#na#,1999-02-02,D4E,D4,E,#na#,#na#,#na#,"Track Type Tractor, Dozer - 75.0 to 85.0 Horsepower",Florida,TTT,Track Type Tractors,#na#,OROPS,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,#na#,#na#,False,True,1328622.0,16000.0,16000.0,1148391.0,4104.0,2.0,0.0,16000.0,16000.0
4,132,2006,#na#,2009-02-05,420D,420,D,#na#,#na#,#na#,Backhoe Loader - 14.0 to 15.0 Ft Standard Digging Depth,Florida,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Standard,Standard,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,True,1560601.0,40000.0,40000.0,1158903.0,3542.0,2.0,0.0,40000.0,40000.0
5,136,1000,#na#,2010-06-26,D4CIIILGP,D4,C,III,LGP,#na#,"Track Type Tractor, Dozer - 75.0 to 85.0 Horsepower",Alaska,TTT,Track Type Tractors,#na#,OROPS,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,None or Unspecified,PAT,None or Unspecified,#na#,#na#,False,False,2264019.0,17000.0,17000.0,632283.0,1545.0,1.0,0.0,17000.0,17000.0
6,132,2006,High,2010-12-09,750J,750,J,#na#,#na#,Medium,"Track Type Tractor, Dozer - 130.0 to 160.0 Horsepower",Georgia,TTT,Track Type Tractors,#na#,EROPS w AC,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,None or Unspecified,PAT,None or Unspecified,#na#,#na#,False,False,1588576.0,85000.0,85000.0,1143493.0,11406.0,1.0,4760.0,85000.0,85000.0
7,132,1986,#na#,2000-04-08,416,416,#na#,#na#,#na#,#na#,Backhoe Loader - 14.0 to 15.0 Ft Standard Digging Depth,Ne

In [ ]:
to.show()

,datasource,UsageBand,saledate,fiModelDesc,fiBaseModel,fiSecondaryDesc,fiModelSeries,fiModelDescriptor,ProductSize,fiProductClassDesc,state,ProductGroup,ProductGroupDesc,Drive_System,Enclosure,Forks,Pad_Type,Ride_Control,Stick,Transmission,Turbocharged,Blade_Extension,Blade_Width,Enclosure_Type,Engine_Horsepower,Hydraulics,Pushblock,Ripper,Scarifier,Tip_Control,Tire_Size,Coupler,Coupler_System,Grouser_Tracks,Hydraulics_Flow,Track_Type,Undercarriage_Pad_Width,Stick_Length,Thumb,Pattern_Changer,Grouser_Type,Backhoe_Mounting,Blade_Type,Travel_Controls,Differential_Type,Steering_Controls,auctioneerID_na,MachineHoursCurrentMeter_na,SalesID,SalePrice,MachineID,ModelID,auctioneerID,YearMade,MachineHoursCurrentMeter,SalePrice
0,121,Low,2006-11-16,521D,521,D,#na#,#na#,#na#,Wheel Loader - 110.0 to 120.0 Horsepower,Alabama,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,1139246,66000,999089,3157,3.0,2004,68.0,66000
1,121,Low,2004-03-26,950FII,950,F,II,#na#,Medium,Wheel Loader - 150.0 to 175.0 Horsepower,North Carolina,WL,Wheel Loader,#na#,EROPS w AC,None or Unspecified,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,23.5,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,Conventional,False,False,1139248,57000,117657,77,3.0,1996,4640.0,57000
2,121,High,2004-02-26,226,226,#na#,#na#,#na#,#na#,Skid Steer Loader - 1351.0 to 1601.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,OROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,1139249,10000,434808,7009,3.0,2001,2838.0,10000
3,121,High,2011-05-19,PC120-6E,PC120,#na#,-6E,#na#,Small,"Hydraulic Excavator, Track - 12.0 to 14.0 Metric Tons",Texas,TEX,Track Excavators,#na#,EROPS w AC,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,2 Valve,#na#,#na#,#na#,#na#,#na#,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,1139251,38500,1026470,332,3.0,2001,3486.0,38500
4,121,Medium,2009-07-23,S175,S175,#na#,#na#,#na#,#na#,Skid Steer Loader - 1601.0 to 1751.0 Lb Operating Capacity,New York,SSL,Skid Steer Loaders,#na#,EROPS,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Auxiliary,#na#,#na#,#na#,#na#,#na#,None or Unspecified,None or Unspecified,None or Unspecified,Standard,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,1139253,11000,1057373,17311,3.0,2007,722.0,11000
5,121,Low,2008-12-18,310G,310,G,#na#,#na#,#na#,Backhoe Loader - 14.0 to 15.0 Ft Standard Digging Depth,Arizona,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,None or Unspecified,No,Extended,Powershuttle,None or Unspecified,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,False,False,1139255,26500,1001274,4605,3.0,2004,508.0,26500
6,121,High,2004-08-26,790ELC,790,E,#na#,LC,Large / Medium,"Hydraulic Excavator, Track - 21.0 to 24.0 Metric Tons",Florida,TEX,Track Excavators,#na#,EROPS,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,Standard,#na#,#na#,#na#,#na#,#na#,None or Unspecified,#na#,#na#,#na#,Steel,None or Unspecified,None or Unspecified,None or Unspecified,None or Unspecified,Double,#na#,#na#,#na#,#na#,#na#,False,False,1139256,21000,772701,1937,3.0,1993,11540.0,21000
7,121,High,2005-11-17,416D,416,D,#na#,#na#,#na#,Backhoe Loader - 14.0 to 15.0 Ft Standard Digging Depth,Illinois,BL,Backhoe Loaders,Four Wheel Drive,OROPS,None or Unspecified,Reversible,No,Standard,Standard,Yes,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#,#na#

In [ ]:
# decode categories
row = to.items.iloc[0]
to.decode_row(row)

In [ ]:
# row, clas, probs = learn.predict(df.iloc[0])

NameError: ignored

# Train

## SKlearn `RandomForestRegressor` Model definition



### First model

In [ ]:
from sklearn.ensemble import RandomForestRegressor
model = RandomForestRegressor(n_estimators=20, max_features=0.3, random_state=0)

### Custom `fit`

In [ ]:
%%time
model.fit(x_train, Y_train)

ValueError: ignored

In [ ]:
predictions = model.predict(X)

### Fast AI `fit`

In [ ]:
X_train_f, y_train_f = to.train.xs, to.train.ys.values.ravel()
X_test_f, y_test_f = to.valid.xs, to.valid.ys.values.ravel()

In [ ]:
%%time
model.fit(X_train_f, y_train_f)

CPU times: user 28.7 s, sys: 11 ms, total: 28.8 s
Wall time: 28.8 s


RandomForestRegressor(bootstrap=True, ccp_alpha=0.0, criterion='mse',
                      max_depth=None, max_features=0.3, max_leaf_nodes=None,
                      max_samples=None, min_impurity_decrease=0.0,
                      min_impurity_split=None, min_samples_leaf=1,
                      min_samples_split=2, min_weight_fraction_leaf=0.0,
                      n_estimators=20, n_jobs=None, oob_score=False,
                      random_state=0, verbose=0, warm_start=False)

# Evaluate

## SKlearn custom

In [ ]:
from sklearn import metrics
from sklearn.metrics import mean_absolute_error

In [ ]:
# get predicted prices on validation data
predicted = model.predict(X_test_f)
print(mean_absolute_error(y_test_f, predicted))

0.011748369689584143


In [ ]:
type(predicted)

numpy.ndarray

In [ ]:
rmse(predicted, y_test_f)

AttributeError: ignored

In [ ]:
# print("Accuracy: %f" % lgs.score(X_test_f, y_test_f))  #checking the accuracy
# print("Precision: %f" % metrics.precision_score(y_test_f, predicted))
# print("Recall: %f" % metrics.recall_score(y_test_f, predicted))
# print("F1-Score: %f" % metrics.f1_score(y_test_f, predicted))
# print("AUC: %f" metrics.auc(fpr, tpr))

ValueError: ignored

# Pipeline

## Sklearn Imports

In [ ]:
from sklearn.compose import ColumnTransformer
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import OneHotEncoder
from sklearn.ensemble import RandomForestRegressor

## processing

In [ ]:
# Preprocessing for numerical data
numerical_transformer = SimpleImputer(strategy='constant')

# Preprocessing for categorical data
categorical_transformer = Pipeline(steps=[
    ('imputer', SimpleImputer(strategy='most_frequent')),
    ('onehot', OneHotEncoder(handle_unknown='ignore'))
])

# Bundle preprocessing for numerical and categorical data
preprocessor = ColumnTransformer(
    transformers=[
        ('num', numerical_transformer, numerical_cols),
        ('cat', categorical_transformer, categorical_cols)
    ])

In [ ]:
model = RandomForestRegressor(n_estimators=20, random_state=0)

In [ ]:
from sklearn.metrics import mean_absolute_error

# Bundle preprocessing and modeling code in a pipeline
my_pipeline = Pipeline(steps=[('preprocessor', preprocessor),
                              ('model', model)
                             ])

# Preprocessing of training data, fit model 
my_pipeline.fit(X_train, y_train)

# Preprocessing of validation data, get predictions
preds = my_pipeline.predict(X_valid)

# Evaluate the model
score = mean_absolute_error(y_valid, preds)
print('MAE:', score)